In [1]:
import tensorflow as tf
from tensorflow.python import keras
import tensorflow_graphics as tfg
import tensorflow_addons as tfa
import tensorflow_probability as tfp
from image_interpolation import ImageInterpolator, ImageSectionRNNCell, generate_2d_grid


2022-09-30 11:22:05.659911: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-30 11:22:05.788667: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-30 11:22:06.443726: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-30 11:22:06.443784: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
starts=tf.keras.Input(shape=(2,))
ends=tf.keras.Input(shape=(2,))
generate_2d_grid(starts, ends, (16, 20))

2022-09-30 11:22:11.191709: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-30 11:22:11.191793: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ideapad5pro): /proc/driver/nvidia/version does not exist
2022-09-30 11:22:11.193544: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<KerasTensor: shape=(None, 16, 20, 2) dtype=float32 (created by layer 'tf.stack_2')>

# Test differentiable batched interpolation

In [3]:
images= tf.keras.Input(shape=(28, 28, 3), name='images')
queries = tf.keras.Input(shape=(16, 16, 2), name='queries')
interpolated = tfp.math.batch_interp_regular_nd_grid(
            queries, [0.0, 0.0], [1.0, 1.0], images, axis=-3)
print(interpolated.shape)

(None, 16, 16, 3)


In [27]:
starts=tf.keras.Input(shape=(2,))
ends=tf.keras.Input(shape=(2,))
# starts=tf.Variable([[0.0, 2.0],[0.0, 3.0]])
# ends=tf.Variable([[1.0, 4.0],[1.0, 5.0]])
nums=[5,7]
dim0_range=tf.linspace(starts[:,0],ends[:,0],nums[0],axis=1)

print("lower range dim 0: ",dim0_range)
second_spacial_dim_start=tf.einsum('i,j->ij',starts[:,1],tf.ones((nums[0],)))
print("lower range second dim: ",second_spacial_dim_start)
lower_stacked=tf.stack([dim0_range,second_spacial_dim_start],axis=2)
print("stacked: ",lower_stacked)
second_spacial_dim_end=tf.einsum('i,j->ij',ends[:,1],tf.ones((nums[0],)))
upper_stacked=tf.stack([dim0_range,second_spacial_dim_end],axis=2)
print("upper stacked: ",upper_stacked)

ranges=[(1.0-alpha)*lower_stacked+alpha*upper_stacked for alpha in [i/(nums[1]-1) for i in range(nums[1])]]
print("ranges:",ranges)
meshed_stacked=tf.stack(ranges,axis=2)
print("stacked:",meshed_stacked)

lower range dim 0:  KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.float32, name=None), name='tf.linspace_14/linspace/Slice:0', description="created by layer 'tf.linspace_14'")
lower range second dim:  KerasTensor(type_spec=TensorSpec(shape=(None, 5), dtype=tf.float32, name=None), name='tf.einsum_2/einsum/Einsum:0', description="created by layer 'tf.einsum_2'")
stacked:  KerasTensor(type_spec=TensorSpec(shape=(None, 5, 2), dtype=tf.float32, name=None), name='tf.stack_11/stack:0', description="created by layer 'tf.stack_11'")
upper stacked:  KerasTensor(type_spec=TensorSpec(shape=(None, 5, 2), dtype=tf.float32, name=None), name='tf.stack_12/stack:0', description="created by layer 'tf.stack_12'")
ranges: [<KerasTensor: shape=(None, 5, 2) dtype=float32 (created by layer 'tf.__operators__.add_12')>, <KerasTensor: shape=(None, 5, 2) dtype=float32 (created by layer 'tf.__operators__.add_13')>, <KerasTensor: shape=(None, 5, 2) dtype=float32 (created by layer 'tf.__operators__.a

In [8]:
inp=tf.keras.Input(shape=(2,))
interval=tf.linspace(inp[:,0],inp[:,1],10)
print(interval)
s=tf.reduce_sum(interval,axis=0)
print(s)
model=tf.keras.Model(inputs=inp,outputs=s)
#model.summary()
values=tf.Variable([[0.0,1.0],[1.0,2.0]])
with tf.GradientTape() as tape:
    tape.watch(values)
    out=model(values)
    print('output:',out)
    print(tape.gradient(out,values))

KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.float32, name=None), name='tf.linspace_10/linspace/Slice:0', description="created by layer 'tf.linspace_10'")
KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name=None), name='tf.math.reduce_sum/Sum:0', description="created by layer 'tf.math.reduce_sum'")
array([[0., 1.],
       [1., 2.]], dtype=float32)>. This is a strong indication that the Lambda layer should be rewritten as a subclassed Layer.
array([[0., 1.],
       [1., 2.]], dtype=float32)>. This is a strong indication that the Lambda layer should be rewritten as a subclassed Layer.
output: tf.Tensor([ 5. 15.], shape=(2,), dtype=float32)
tf.Tensor(
[[5. 5.]
 [5. 5.]], shape=(2, 2), dtype=float32)


In [5]:
interpolatorImageInput=tf.keras.Input(shape=(28, 28, 1))
interpolatorSectionInput=tf.keras.Input(shape=(3,))
print(tfg.geometry.representation.grid._grid(starts=[1.0, 1.0], stops=[2.0, 2.0], nums=[3, 3]))
interpolatorOutput=ImageInterpolator(grid_dim=(16, 16))(interpolatorImageInput,interpolatorSectionInput)

tf.Tensor(
[[[1.  1. ]
  [1.  1.5]
  [1.  2. ]]

 [[1.5 1. ]
  [1.5 1.5]
  [1.5 2. ]]

 [[2.  1. ]
  [2.  1.5]
  [2.  2. ]]], shape=(3, 3, 2), dtype=float32)
(<tf.Tensor 'image_interpolator_1/add:0' shape=(None, 2) dtype=float32>,)


ValueError: Exception encountered when calling layer "image_interpolator_1" (type ImageInterpolator).

in user code:

    File "/home/olep/Documents/software/continousSpaceRNN/image_interpolation.py", line 33, in call  *
        queries = generate(starts, stops, self.grid_dim)
    File "/home/olep/Documents/software/continousSpaceRNN/image_interpolation.py", line 93, in generate  *
        grid._grid(starts, stops, nums)
    File "/home/olep/.local/lib/python3.10/site-packages/tensorflow_graphics/geometry/representation/grid.py", line 46, in _grid  *
        params = [tf.unstack(tensor) for tensor in [starts, stops, nums]]

    ValueError: Cannot infer argument `num` from shape (None, 2)


Call arguments received by layer "image_interpolator_1" (type ImageInterpolator):
  • image=tf.Tensor(shape=(None, 28, 28, 1), dtype=float32)
  • section=tf.Tensor(shape=(None, 3), dtype=float32)

In [8]:
gridInput=tf.keras.Input(shape=(3,))
grid=tfg.geometry.representation.grid.generate(starts=[1.0, 1.0], stops=[2.0, 2.0], nums=[3, 3])
print(grid)

ValueError: Argument `axis` = 0 not in range [0, 0)

## Try RNN

In [3]:
input=tf.keras.Input(shape=(28, 28, 1))
num_iterations=20
repeated_input = tf.repeat(tf.expand_dims(input,1), repeats=num_iterations, axis=1)
print(repeated_input.shape)
classifier=keras.Sequential([
    tf.keras.layers.RNN(ImageSectionRNNCell(grid_dim=(8, 8), units=32)),
    tf.keras.layers.Dense(10, activation='softmax')
])
output=classifier(repeated_input)
model=tf.keras.Model(inputs=input, outputs=output)
model.summary()

2022-09-27 14:16:07.211653: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-27 14:16:07.211762: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ideapad5pro): /proc/driver/nvidia/version does not exist
2022-09-27 14:16:07.213656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(None, 20, 28, 28, 1)
(<tf.Tensor 'module_wrapper/rnn/image_section_rnn_cell/image_interpolator_1/add:0' shape=(None, 2) dtype=float32>,)
queries:  Tensor("module_wrapper/rnn/image_section_rnn_cell/image_interpolator_1/stack:0", shape=(None, 1), dtype=float32)
(<tf.Tensor 'rnn/image_section_rnn_cell/image_interpolator_1/add:0' shape=(None, 2) dtype=float32>,)
queries:  Tensor("rnn/image_section_rnn_cell/image_interpolator_1/stack:0", shape=(None, 1), dtype=float32)


ValueError: Exception encountered when calling layer "image_section_rnn_cell" "                 f"(type ImageSectionRNNCell).

in user code:

    File "/home/olep/Documents/software/continousSpaceRNN/image_interpolation.py", line 73, in call  *
        processed_section_values = self.section_processor(section_values)
    File "/home/olep/.local/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/home/olep/.local/lib/python3.10/site-packages/keras/engine/input_spec.py", line 250, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception encountered when calling layer "sequential" "                 f"(type Sequential).
    
    Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, None, None)
    
    Call arguments received by layer "sequential" "                 f"(type Sequential):
      • i

## Experiments with the grid interpolations

In [ ]:
queries = tf.Variable([[[1.1, 1.3]]])
with tf.GradientTape() as tape:
    z = tfp.math.batch_interp_regular_nd_grid(
        queries, [0.0, 0.0], [3.0, 3.0], image, axis=-3)
    print(z)
print(tape.gradient(z, queries))


tf.Tensor([[[4.0666666]]], shape=(1, 1, 1), dtype=float32)
tf.Tensor([[[2.0000002 0.6666665]]], shape=(1, 1, 2), dtype=float32)


In [2]:
image = tf.constant(
    [[[[1.0], [2.0], [3.0]], [[4.0], [5.0], [6.0]], [[7.0], [8.0], [9.0]]]])
interpolator = ImageInterpolator(grid_dim=(3, 3))
print('Interpolation: ', interpolator(image, tf.constant([[0.0, 0.0, 1.0]])))
section = tf.Variable([[0.0, 0.0, 1.0]])
with tf.GradientTape() as tape:
    interpolation = interpolator(image, section)
    print('Interpolation: ', interpolation)
    # Take sum alon the last two axes
    s = tf.reduce_sum(interpolation, axis=1)
    s2 = tf.reduce_sum(s, axis=1)
    print("Sum: ", s2)
print("Gradient of sum w.r.t. section:", tape.gradient(s2, section))


(<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1., 1.]], dtype=float32)>,)
queries:  tf.Tensor(
[[0. ]
 [0. ]
 [0.5]
 [0.5]
 [1. ]
 [1. ]], shape=(6, 1), dtype=float32)
Interpolation:  tf.Tensor(
[[[1.]
  [1.]
  [5.]
  [5.]
  [9.]
  [9.]]], shape=(1, 6, 1), dtype=float32)
(<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1., 1.]], dtype=float32)>,)
queries:  tf.Tensor(
[[0. ]
 [0. ]
 [0.5]
 [0.5]
 [1. ]
 [1. ]], shape=(6, 1), dtype=float32)
Interpolation:  tf.Tensor(
[[[1.]
  [1.]
  [5.]
  [5.]
  [9.]
  [9.]]], shape=(1, 6, 1), dtype=float32)
Sum:  tf.Tensor([30.], shape=(1,), dtype=float32)
Gradient of sum w.r.t. section: tf.Tensor([[24. 24. 24.]], shape=(1, 3), dtype=float32)


2022-09-27 14:27:30.296973: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-27 14:27:30.297022: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ideapad5pro): /proc/driver/nvidia/version does not exist
2022-09-27 14:27:30.299422: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
startsInput=tf.keras.Input(shape=(2,))

stopsInput=tf.keras.Input(shape=(2,))
tfg.geometry.representation.grid.generate(starts=startsInput, stops=stopsInput, nums=[3, 3])

ValueError: Exception encountered when calling layer "tf.unstack_1" (type TFOpLambda).

Cannot infer argument `num` from shape (None, 2)

Call arguments received by layer "tf.unstack_1" (type TFOpLambda):
  • value=tf.Tensor(shape=(None, 2), dtype=float32)
  • num=None
  • axis=0
  • name=unstack